In [1]:
import os
from dotenv import load_dotenv
import os

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

In [2]:
params = GenerateParams(
            decoding_method="greedy",
            max_new_tokens=1000,
            min_new_tokens=200,
            temperature=0.7,
        )

In [3]:
llm = LangChainInterface(model="ibm/granite-13b-sft", params=params, credentials=creds)

In [4]:
def load_pdf(): 
    pdf_name = 'test.pdf'
    loaders = [PyPDFLoader(pdf_name)]

    index = VectorstoreIndexCreator(
        embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2'), 
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    ).from_loaders(loaders)

    return index

In [5]:
index = load_pdf()

chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=index.vectorstore.as_retriever(), input_key='question')

/Users/charanahu/VS_Code/YouTube/watsonxRAG/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
response = chain.run("Summarize the document.")

In [7]:
print(response)

 This document describes the process of training an LLM to generate summaries of news articles. The model is based on GPT-3 and is trained on a large corpus of news articles. The model is evaluated on a test set of articles and the summaries generated by the model are compared to the summaries generated by a human. The model is found to be accurate and efficient and is able to generate summaries that are similar to human summaries. The model is also found to be able to generate summaries that are coherent and logical. The model is able to generate summaries that are coherent and logical because it is trained on a large corpus of news articles. The model is also found to be accurate and efficient because it is trained on a large corpus of news articles. The model is also found to be able to generate summaries that are similar to human summaries because it is trained on a large corpus of news articles. The model is also found to be accurate and efficient because it is trained on a large 